# Pollard's algorithm for discrete logarithms


This program was made for an assignment of the class ”Cryptography” of my master’s program:  
"Use the Pollard algorithm to calculate the discrete logarithm $x$ inside $\mathbb{Z}_N=\mathbb{Z}_{1693}$, where $17^x \equiv 101 \, mod \, 1693$."

<u>**Pollard's algorithm**</u>  
$\cdot$ Input: A cyclic group $G=<g>$ with $|G|=n>1$ and $\beta \in G$ with $\beta \neq 1$
$\cdot$ Output: $x=\log_g \beta$
1. We choose three non-empty subsets $S_1, S_2, S_3$ of $G$ such that $S_1 \cup S_2 \cup S_3 = G$, $1 \not\in S_2$ and $S_i \cap S_j = \emptyset$, for $i \neq j$, and define the mapping $f: G \times \mathbb{Z}_n \times \mathbb{Z}_n \rightarrow G \times \mathbb{Z}_n \times \mathbb{Z}_n$ with
\begin{equation*}
f(x,a,b)=\begin{cases} (\beta x \, mod \, N, \; \; \; a \, mod \, n \; \; \; \; , (b+1) \, mod \, n), \text{ if } x \in S_1 \\
      (x^2 \, mod \, N, \; \; \; 2a \, mod \, n \; \; \; ,  \; \; \;  2b \, mod \, n  \; \; \; ), \text{ if } x \in S_2 \\
      (gx \, mod \, N, (a+1) \, mod \, n,  \; \; \;  b \, mod \, n  \; \; \; \;), \text{ if } x \in S_3 \end{cases} \end{equation*}
2. We define the sequence 
\begin{equation*}
(x_i, a_i, b_i) = \begin{cases} \; \; \; \; \; \; \;  (1,0,0) \; \; \; \; \; \; \; , \text{ if } i=0 \\
                                f(x_{i-1}, a_{i-1}, b_{i-1}), \text{ if } i \geq 1 \end{cases}
\end{equation*}
3. We compare the trios $(x_i, a_i, b_i)$ and $(x_{2i}, a_{2i}, b_{2i})$ until we find an index $i \geq 1$ such that $x_{2i}=x_i$.
4. We solve the modular equation $(b_{2i}-b_i)z \equiv (a_i-a_{2i}) \, [mod \, n]$.
5. One of the solutions of the above equality is the discrete logarithm $\log_g \beta$.

**Note**: In order to have some control over the iterations we will also add an index $i_{upper}$ which will limit the number of trios that we will create and compare.  
                       
Author: Florias Papadopoulos

## Importing modules

We start by importing the modules that we will use

In [1]:
import math

## Defining the functions

### Starter functions

#### (a) func

First, we recreate the function $f$ of the algorithm in python.

In [2]:
def func(N,n,g,beta,x,a,b,S_1,S_2,S_3):

  #upper case
  if x in S_1:
        one = (beta*x) % N
        two = a % n
        three = (b+1) % n
        return one, two, three

  #middle case
  if x in S_2:
        one = (x**2) % N
        two = (2*a) % n
        three = (2*b) % n
        return one, two, three

  #bottom case
  if x in S_3:
        one = (g*x) % N
        two = (a+1) % n
        three = b % n
        return one, two, three

#### (b) tri_seq

This function will be used to create the trios $(x_i, a_i, b_i)$ until the chosen $i=i_{upper}$.

In [3]:
def tri_seq(N,n,g,beta,S_1,S_2,S_3,i_upper):

    tri_seq_list = [[1,0,0]]
    x_i, a_i, b_i = 1, 0, 0
    for i in range(0,i_upper+1):
        tri_seq_sub_list = []
        (x_next, a_next, b_next) = func(N,n,g,beta,x_i,a_i,b_i,S_1,S_2,S_3)
        tri_seq_sub_list.append(x_next)
        tri_seq_sub_list.append(a_next)
        tri_seq_sub_list.append(b_next)
        tri_seq_list.append(tri_seq_sub_list)
        x_i = x_next
        a_i = a_next
        b_i = b_next
    return tri_seq_list

#### (c) modulo_solver

A function that will help us solve the modular equation of the 4th step of the algorithm.  
More precisely, this function solves the modular equation $bx \equiv a \, (mod \, n)$.

In [4]:
def modulo_solver(b,a,n):
    
    #print(b,"* x =",a,"(modn)")
    #print("=>")
    
    x_list = []
    if b != 0 and b == a:
        x_list.append(1)
    if b != 0 and b !=a :
        gcd = math.gcd(math.gcd(b,a),n)
        b_gcd = b//gcd
        a_gcd = a//gcd
        n_gcd = n//gcd
        
        #print(b_gcd,"* x =",a_gcd,"(",n_gcd,")")
        if b_gcd == 1:
            x = a_gcd % n_gcd
        if b_gcd == -1:
            x = (-1)*a_gcd % n_gcd
        if b_gcd != 1 and b_gcd !=-1:
            b_inv = pow(b_gcd, -1,n_gcd)
            x = (b_inv*a_gcd) % n_gcd
    
        for k in range(0,gcd):
            x_k = k*n_gcd + x
            x_list.append(x_k)
            
    return x_list

#### (d) verify

Lastly, we will need a function that can check if the solution of the modular equation that was found on the 5th step is indeed $\log_g \beta$.  
That's what "verify" does.

In [5]:
def verify(N,g,beta,solution):
    return pow(g,solution,N) == beta

### discr_pollardAlgorithm

We will now create our main function based on the Pollard algorithm, albeit with a twist.  
In particular, as we mentioned before, we will input an $i_{upper}$ that controls the number of trios that will be created.
Moreover, our algorithm will not only output the discrete logarithm, but all values that were calculated in each step.

In [6]:
def discr_pollardAlgorithm(N,n,g,beta,i_upper):
    
    #error message in case i_upper is not enough
    proper_solution = "problem, increase i_upper"

    #creating S_1,S_2,S_3
    S_1 = set()
    S_2 = set()
    S_3 = set()

    for x in range(1,N):
        if (x-1) % 3 == 0:
            S_1.add(x)
        if x % 3 == 0:
            S_2.add(x)
        if (x-2) % 3 == 0:
            S_3.add(x)

    #creating tri_seq
    seq = tri_seq(N,n,g,beta,S_1,S_2,S_3,i_upper)

    #finding the solutions z
    z_list = []

    for t in range(1,math.floor(i_upper/2)):
        [x_t, a_t, b_t] = seq[t]
        [x_2t, a_2t, b_2t] = seq[2*t]

        if x_t == x_2t:
            b_minus = b_2t - b_t
            a_minus = a_t - a_2t
            z_list = modulo_solver(b_minus,a_minus,n)
            
            #finding and verifying the correct z from the list that contains all of the solutions z
            for z in z_list:
                if verify(N,g,beta,z) == True:
                    proper_solution = z
                    break
            break

    return S_1, S_2, S_3, seq, t, x_t, x_2t, b_t, b_2t, a_t, a_2t, z_list, proper_solution

## Solving the problem

We create a script that uses the above function to return us a text that elaborates on all the values that were computed in each step of the algorithm.  
In our problem $N=1693$, $n=1962$, $g=17$ and $\beta =101$.

In [7]:
#input
N, n, g, beta = 647, 646, 5, 501 #n=N-1 in our case
i_upper = 1000
#input

(S_1, S_2, S_3, seq, t, x_t, x_2t, b_t, b_2t, a_t, a_2t, z_list, proper_solution) = discr_pollardAlgorithm(N,n,g,beta,i_upper)

print("We wanted to find the discrete logarithm log_" +str(g) + "(" + str(beta)+ ") inside Z*_" + str(N) +", with ord_" \
      + str(N) + "(" + str(g) + ")=" + str(n))
print("For this, we used Pollard's Algorithm and did the following:")
print("")
print("")
print("In step 1, we formed the subsets S_1, S_2, S_3 as done in the example in class and defined the function f.")
print("S_1=",S_1)
print("S_2=",S_2)
print("S_3=",S_3)
print("")
print("In step 2, we defined the sequence (x_i,a_i,b_i) for i in [0," + str(i_upper) + "]: ")
print(seq)
print("")
print("In step 3, we compared x_t and x_2t until we found x_t=x_2t=" + str(x_t) + " for t=" + str(t))
print("In step 4, we solved the modular equation " + str(b_2t) + "-" + str(b_t) + "(equiv)" + str(a_t) + "-" + str(a_2t) \
      + "[mod" + str(n) + "]")
if len(z_list) != 1:
    print("and found the set of possible solutions: ", z_list)
    print("Of all of them, we verified that the proper is " + str(proper_solution))
if len(z_list) == 1:
    print("and found the solution " + str(proper_solution) + " which is the discrete logarithm that we were looking for.")

We wanted to find the discrete logarithm log_5(501) inside Z*_647, with ord_647(5)=646
For this, we used Pollard's Algorithm and did the following:


In step 1, we formed the subsets S_1, S_2, S_3 as done in the example in class and defined the function f.
S_1= {1, 514, 4, 517, 7, 520, 10, 523, 13, 526, 16, 529, 19, 532, 22, 535, 25, 538, 28, 541, 31, 544, 34, 547, 37, 550, 40, 553, 43, 556, 46, 559, 49, 562, 52, 565, 55, 568, 58, 571, 61, 574, 64, 577, 67, 580, 70, 583, 73, 586, 76, 589, 79, 592, 82, 595, 85, 598, 88, 601, 91, 604, 94, 607, 97, 610, 100, 613, 103, 616, 106, 619, 109, 622, 112, 625, 115, 628, 118, 631, 121, 634, 124, 637, 127, 640, 130, 643, 133, 646, 136, 139, 142, 145, 148, 151, 154, 157, 160, 163, 166, 169, 172, 175, 178, 181, 184, 187, 190, 193, 196, 199, 202, 205, 208, 211, 214, 217, 220, 223, 226, 229, 232, 235, 238, 241, 244, 247, 250, 253, 256, 259, 262, 265, 268, 271, 274, 277, 280, 283, 286, 289, 292, 295, 298, 301, 304, 307, 310, 313, 316, 319, 322, 325, 328